In [ ]:
import fastf1
import pandas as pd
import os

# --- CONFIGURATION ---
YEARS_TO_FETCH = range(2018, 2026)
# TRACKS = ['Spa-Francorchamps', 'Monza']
TRACKS = ['Monza']

# NOW FETCHING BOTH SESSIONS
# 'Q' = Qualifying (Speed), 'R' = Race (Volume)
SESSIONS = ['Q', 'R']

def get_fastest_lap_telemetry(year, gp_name, session_type):
    print(f"Attempting to fetch: {year} {gp_name} [{session_type}]...")

    try:
        # 1. Load Session
        session = fastf1.get_session(year, gp_name, session_type)
        session.load(telemetry=True, laps=True, weather=False, messages=False)

        # 2. Pick Fastest Lap
        # Even in a race, we pick the single fastest lap to get the "cleanest" data
        fastest_lap = session.laps.pick_fastest()
        driver = fastest_lap['Driver']

        # 3. Get Telemetry
        tel = fastest_lap.get_telemetry()

        # 4. Context Columns
        tel['Year'] = year
        tel['Track'] = gp_name
        tel['Session'] = session_type # Keep track of if it was Q or R
        tel['Driver'] = driver
        tel['Lap_Time'] = str(fastest_lap['LapTime']).split('days')[-1].strip()

        # Row-by-Row Counter
        tel['Sample_Count'] = range(1, len(tel) + 1)

        # Unique ID: Year_Track_Session_Driver_Counter
        # Example: 2023_Spa_R_VER_150
        tel['Point_ID'] = (
            tel['Year'].astype(str) + "_" +
            tel['Track'] + "_" +
            tel['Session'] + "_" +
            tel['Driver'] + "_" +
            tel['Sample_Count'].astype(str)
        )

        # 5. Clean Units
        tel['X'] = tel['X'] / 10
        tel['Y'] = tel['Y'] / 10

        cols_to_keep = [
            'Point_ID',
            'Year', 'Track', 'Session', 'Driver', 'Lap_Time',
            'Time', 'Distance',
            'X', 'Y',
            'Speed', 'Throttle', 'Brake', 'nGear', 'RPM', 'DRS'
        ]

        return tel[cols_to_keep].copy()

    except Exception as e:
        # Quietly skip if data is missing (common for some older sprints/races)
        return None

# --- MAIN EXECUTION ---
if not os.path.exists('cache'):
    os.makedirs('cache')
fastf1.Cache.enable_cache('cache')

all_races_data = []

for year in YEARS_TO_FETCH:
    for track in TRACKS:
        for ses in SESSIONS: # <--- New Inner Loop
            df = get_fastest_lap_telemetry(year, track, ses)

            if df is not None and not df.empty:
                all_races_data.append(df)
                print(f"  -> Success! {year} {track} ({ses}) - Rows: {len(df)}")

if all_races_data:
    final_df = pd.concat(all_races_data, ignore_index=True)

    # --- Per-track normalization: X and Y scaled to [0,1] per Track ---
    # This creates X_norm and Y_norm columns so downstream notebooks can use
    # consistent normalized coordinates across different track shapes/sizes.
    group = final_df.groupby('Track')

    min_x = group['X'].transform('min')
    max_x = group['X'].transform('max')
    dx = max_x - min_x
    # avoid division by zero for degenerate tracks
    dx = dx.replace(0, 1.0)
    final_df['X_norm'] = (final_df['X'] - min_x) / dx

    min_y = group['Y'].transform('min')
    max_y = group['Y'].transform('max')
    dy = max_y - min_y
    dy = dy.replace(0, 1.0)
    final_df['Y_norm'] = (final_df['Y'] - min_y) / dy

    filename = "api_data.csv"
    final_df.to_csv(filename, index=False)

    # Sanity checks
    print(final_df.groupby('Track').agg(
        X_min=('X','min'), X_max=('X','max'),
        Y_min=('Y','min'), Y_max=('Y','max')
    ))
    print(final_df.groupby('Track').agg(
        X_norm_min=('X_norm','min'), X_norm_max=('X_norm','max'),
        Y_norm_min=('Y_norm','min'), Y_norm_max=('Y_norm','max')
    ))

    print(f"\nSaved to {filename} with {len(final_df)} rows.")
else:
    print("No data.")

Attempting to fetch: 2018 Monza [Q]...


core           INFO 	Loading data for Italian Grand Prix - Qualifying [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	No cached data found for session_status_data. Loading data

  -> Success! 2018 Monza (Q) - Rows: 595
Attempting to fetch: 2018 Monza [R]...


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching

  -> Success! 2018 Monza (R) - Rows: 627
Attempting to fetch: 2019 Monza [Q]...


...

  -> Success! 2024 Monza (R) - Rows: 626
Attempting to fetch: 2025 Monza [Q]...


core           INFO 	Loading data for Italian Grand Prix - Qualifying [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	No cached data found for session_status_data. Loading data

  -> Success! 2025 Monza (Q) - Rows: 587
Attempting to fetch: 2025 Monza [R]...


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching

  -> Success! 2025 Monza (R) - Rows: 602
            X_min   X_max       Y_min        Y_max
Track                                             
Monza -150.008815  1107.0 -580.582445  1590.505704
       X_norm_min  X_norm_max  Y_norm_min  Y_norm_max
Track                                                
Monza         0.0         1.0         0.0         1.0

Saved to api_data.csv with 10159 rows.
